In [164]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

import glob
import os
import os.path as osp
import shutil
import random
import math
import json
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import models, transforms

In [165]:
class_index = ['コンクリート', 'レール',  '背景あり', '金枠あり']
weight_index = [0, 0, 0, 0]
batch_size = 32

データの前処理

In [166]:
class makeDataList():
    def __init__(self):
        self.rootpath =  './ImageData/'
        # 10枚に１枚が検証用データ
        self.trainPer = 10
    
    def shuffle_train_test(self):
        for path in glob.glob(self.rootpath+'/*'):
            print(path)

            trainpath = './train/'+path.split('/')[2]
            if not os.path.exists(trainpath):
                os.makedirs(trainpath)

            valpath = './val/'+path.split('/')[2]
            if not os.path.exists(valpath):
                os.makedirs(valpath)

            for j,image in enumerate(glob.glob(self.rootpath+path.split('/')[2]+'/*.jpg')):
                print(j)
                if j%self.trainPer==0:
                    shutil.copyfile(image, './val/'+path.split('/')[2]+'/'+image.split('/')[3])
                else:
                    shutil.copyfile(image, './train/'+path.split('/')[2]+'/'+image.split('/')[3])
                    
    def make_data_path(self, phase='train'):
        target_path = osp.join('./'+phase+'/**/*.jpg')
        path_list = []
        for path in glob.glob(target_path):
            path_list.append(path)
        
        return path_list

In [167]:
class BaseTransform():
    def __init__(self):
        self.size = 256
        self.mean = ((0.5,))
        self.std = ((0.22,))
        self.base_transform = {
            'train':
                transforms.Compose([
                transforms.RandomResizedCrop(self.size, scale=(0.5,1.0)),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(self.mean, self.std)
                ]),
            'val':
                transforms.Compose([
                transforms.Resize(self.size),
                transforms.CenterCrop(self.size),
                transforms.ToTensor(),
                transforms.Normalize(self.mean, self.std)
                ])
        }
        
    def __call__(self, img, phase='train'):
        return self.base_transform[phase](img)

In [168]:
class HymenopteraDataset(data.Dataset):
    
    def __init__(self, file_list, transform=None, phase='train'):
        self.file_list = file_list
        self.transform = transform
        self.phase = phase
        
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, index):
        img_path = self.file_list[index]
        img = Image.open(img_path)
        img_transformed = self.transform(img, self.phase)
        if self.phase == 'train':
            label = img_path.split('/')[2]
            label = class_index.index(label)
        elif self.phase == 'val':
            label = img_path.split('/')[2]
            label = class_index.index(label)
        
        return img_transformed, label

In [169]:
dataList = makeDataList()
#dataList.shuffle_train_test()
trainList = dataList.make_data_path(phase='train')
valList = dataList.make_data_path(phase='val')

In [170]:
trainDataset = HymenopteraDataset(file_list=trainList, transform=BaseTransform(), phase='train')
valDataset = HymenopteraDataset(file_list=valList, transform=BaseTransform(), phase='val')

In [171]:
index = 10
print(trainDataset.__getitem__(index)[0].size())
print(trainDataset.__getitem__(index)[1])

torch.Size([1, 256, 256])
0


In [172]:
for i,class_ in enumerate(class_index):
    files = glob.glob('./train/'+class_+'/*.jpg')
    weight_index[i] = len(files)
weight_index

[2459, 1375, 1891, 251]

In [173]:
trainDataloader = torch.utils.data.DataLoader(trainDataset,
                                              batch_size=batch_size,
                                              shuffle=True)
valDataloader = torch.utils.data.DataLoader(valDataset,
                                            batch_size=batch_size,
                                            shuffle=False)
dataloaders_dict = { 'train': trainDataloader, 'val': valDataloader }
batch_iterator = iter(dataloaders_dict['train'])
inputs, labels = next(batch_iterator)

print(inputs.size())
print(labels)

torch.Size([32, 1, 256, 256])
tensor([0, 0, 1, 2, 0, 0, 0, 3, 1, 0, 2, 1, 3, 0, 0, 0, 2, 0, 2, 3, 1, 1, 2, 1,
        0, 1, 0, 1, 2, 0, 1, 1])


In [174]:
new_weight_index = list(map(lambda x: float(math.ceil(sum(weight_index)/x)), weight_index))
new_weight_index  = list(map(lambda x: x/3, new_weight_index))

In [175]:
new_weight_index = [1.0, 1.7, 1.7, 8.0]

GoogLeNetを使った学習

In [176]:
class MyNet(nn.Module):
    def __init__(self):
        super(MyNet,self).__init__()
        self.conv1 = nn.Conv2d(1,32,3,1)
        self.conv2 = nn.Conv2d(32,64,3,1)
        self.pool = nn.MaxPool2d(2,2)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(12*12*64,128)
        self.fc2 = nn.Linear(128,10)

    def forward(self,x):
        x = self.conv1(x)
        x = f.relu(x)
        x = self.conv2(x)
        x = f.relu(x)
        x = self.pool(x)
        x = self.dropout1(x)
        x = x.view(-1,12*12*64)
        x = self.fc1(x)
        x = f.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)

        return f.log_softmax(x, dim=1)

In [181]:
class ResNet():
    
    def __init__(self):
        self.net = models.resnet18(pretrained=True)
        self.net.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.net.fc = nn.Linear(self.net.fc.in_features, len(class_index))
        self.net.train()
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.net.to(self.device)
        self.num_epochs = 50
        print('use in device: ', self.device)
        print(self.net.fc.out_features)
        weights = torch.tensor(new_weight_index).cuda()
        print(weights)
        self.criterion = nn.CrossEntropyLoss(weight=weights)
        self.optimizer = optim.SGD(params=self.net.parameters(), lr=0.001, momentum=0.9)
        self.lr_scheduler = optim.lr_scheduler.StepLR(self.optimizer, step_size=7, gamma=0.1)
        self.bestAcc = 0.0
#         load_path = './weightsFineTuningColor_epoch48_0.724258.pth'
#         load_weights = torch.load(load_path)
#         self.net.load_state_dict(torch.load(load_path, map_location={'cuda:0': 'cpu'}))
        
    def train(self, dataloaderDict):
        self.net.to(self.device)
        torch.backends.cudnn.benchmark = True
        for epoch in range(self.num_epochs):
            print('Epochs {}/{}'.format(epoch+1, self.num_epochs))
            for phase in ['train', 'val']:
                if phase == 'train':
                    self.lr_scheduler.step()
                    self.net.train()
                else:
                    self.net.eval()
                    
                epoch_loss = 0.0
                epoch_corrects = 0
                
                if (epoch == 0) and (phase == 'train'):
                    continue
                for inputs, labels in tqdm(dataloaderDict[phase]):
                    inputs, labels = inputs.to(self.device), labels.to(self.device)
                    outputs = self.net(inputs)
                    loss = self.criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)
                    
                    if phase == 'train':
                        loss.backward()
                        self.optimizer.step()
                        
                    epoch_loss += loss.item() * inputs.size(0)
                    epoch_corrects += torch.sum(preds == labels.data)
                    
                epoch_loss = epoch_loss / len(dataloaderDict[phase].dataset)
                epoch_acc = epoch_corrects.double() / len(dataloaderDict[phase].dataset)
                if( phase=='val') and ( 0.7 < epoch_acc ):
                    self.saveWeight(epoch, epoch_acc)
                    self.bestAcc = epoch_acc
                    print('save weight')
                print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
                
    def eval(self, imgList, transform):
        
        
        for label in class_index:
            path = './result/'+label
            if not os.path.exists(path):
                os.makedirs(path)
        
        load_path = './weightsFineTuningColor_epoch48_0.724258.pth'
        load_weights = torch.load(load_path)
        print(load_path)
        
        self.net.load_state_dict(load_weights)
        self.net.load_state_dict(torch.load(load_path, map_location={'cuda:0': 'cpu'}))
        self.net.eval()
        
        for _img in imgList:
            img = Image.open(_img)
            img_transformed = transform(img)
            inputs = img_transformed.unsqueeze_(0)
            inputs = inputs.to(self.device)
            out = self.net(inputs)
            img.close()
            print(out)
            maxid = np.argmax(out.cpu().detach().numpy())
            predict_label_name = class_index[maxid]
            print(predict_label_name)
            filename = _img.split('/')[-1]
            shutil.copyfile(_img, './result/'+ predict_label_name+'/'+filename)
            
                
    def saveWeight(self, i, acc):
        save_path = './weightsFineTuningColor_epoch%d_%f.pth'%(i,acc)
        torch.save(self.net.state_dict(), save_path)





 47%|████▋     | 88/187 [00:26<00:09, 10.75it/s]

In [ ]:
resnetTrain = ResNet()
resnetTrain.train(dataloaders_dict)






  0%|          | 0/4 [00:00<?, ?it/s]




100%|██████████| 4/4 [00:00<00:00, 22.17it/s]





  0%|          | 0/187 [00:00<?, ?it/s]

use in device:  cuda:0
4
tensor([1.0000, 1.7000, 1.7000, 8.0000], device='cuda:0')
Epochs 1/50
val Loss: 1.3553 Acc: 0.4160
Epochs 2/50







  1%|          | 1/187 [00:00<00:18,  9.91it/s]




  2%|▏         | 3/187 [00:00<00:18, 10.06it/s]




  3%|▎         | 5/187 [00:00<00:17, 10.24it/s]




  4%|▎         | 7/187 [00:00<00:17, 10.43it/s]




  5%|▍         | 9/187 [00:00<00:16, 10.53it/s]




  6%|▌         | 11/187 [00:01<00:16, 10.55it/s]




  7%|▋         | 13/187 [00:01<00:16, 10.51it/s]




  8%|▊         | 15/187 [00:01<00:16, 10.56it/s]




  9%|▉         | 17/187 [00:01<00:15, 10.67it/s]




 10%|█         | 19/187 [00:01<00:15, 10.75it/s]




 11%|█         | 21/187 [00:01<00:15, 10.67it/s]




 12%|█▏        | 23/187 [00:02<00:15, 10.71it/s]




 13%|█▎        | 25/187 [00:02<00:15, 10.64it/s]




 14%|█▍        | 27/187 [00:02<00:15, 10.43it/s]




 16%|█▌        | 29/187 [00:02<00:15, 10.41it/s]




 17%|█▋        | 31/187 [00:02<00:14, 10.42it/s]




 18%|█▊        | 33/187 [00:03<00:14, 10.48it/s]




 19%|█▊        | 35/187 [00:03<00:14, 10.28it/s]




 20%|█▉        | 37/187 [00:03<00:14, 10.44it/

テスト

In [126]:
class MakeTestDataList():
    
    def __init__(self):
        self.rootpath =  './Test/'
        self.dir = []
        self.datapath = []
        
    def getTest(self):
        for dirpath in glob.glob(self.rootpath+'/*'):
            self.dir.append(dirpath)
            for path in glob.glob(dirpath+'/*.jpg'):
                self.datapath.append(path)
                
        return self.datapath
    

In [127]:
class TestBaseTransform():
    def __init__(self):
        self.size = 256
        self.mean = ((0.5,))
        self.std = ((0.22,))
        self.base_transform = {
            'test':
                transforms.Compose([
                transforms.RandomResizedCrop(self.size, scale=(0.5,1.0)),
                transforms.ToTensor(),
                transforms.Normalize(self.mean, self.std)
                ]),
        }
        
    def __call__(self, img, phase='test'):
        return self.base_transform[phase](img)

In [130]:
resnetTrain = ResNet()
test = MakeTestDataList()
path = test.getTest()
transform = TestBaseTransform()
resnetTrain.eval(path, transform)

use in device:  cuda:0
4
tensor([ 3.,  6.,  4., 23.], device='cuda:0')
./weightsFineTuningColor_epoch48_0.724258.pth
tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       grad_fn=<AddmmBackward>)
コンクリート
tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       grad_fn=<AddmmBackward>)
コンクリート
tensor([[-30.8573,  -5.3766,  20.8814,  15.3734]], device='cuda:0',
       grad_fn=<AddmmBackward>)
背景あり
tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       grad_fn=<AddmmBackward>)
コンクリート
tensor([[-30.9879,  -5.3840,  20.9388,  15.4543]], device='cuda:0',
       grad_fn=<AddmmBackward>)
背景あり
tensor([[-5.1329, -3.9315,  9.5801, -0.5625]], device='cuda:0',
       grad_fn=<AddmmBackward>)
背景あり
tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       grad_fn=<AddmmBackward>)
コンクリート
tensor([[-32.5175,  -5.4171,  21.7864,  16.1716]], device='cuda:0',
       grad_fn=<AddmmBackward>)
背景あり
tensor([[-12.6879,  -4.3559,  12.8992,   4.1177]], devi

tensor([[-43.9073,  -6.1097,  26.6146,  23.4576]], device='cuda:0',
       grad_fn=<AddmmBackward>)
背景あり
tensor([[-39.2010,  -5.8453,  24.5470,  20.5422]], device='cuda:0',
       grad_fn=<AddmmBackward>)
背景あり
tensor([[-17.0442,  -4.5382,  14.7115,   6.8586]], device='cuda:0',
       grad_fn=<AddmmBackward>)
背景あり
tensor([[-13.5705,  -4.4048,  13.2858,   4.6649]], device='cuda:0',
       grad_fn=<AddmmBackward>)
背景あり
tensor([[-27.3597,  -5.1801,  19.3449,  13.2067]], device='cuda:0',
       grad_fn=<AddmmBackward>)
背景あり
tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       grad_fn=<AddmmBackward>)
コンクリート
tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       grad_fn=<AddmmBackward>)
コンクリート
tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       grad_fn=<AddmmBackward>)
コンクリート
tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       grad_fn=<AddmmBackward>)
コンクリート
tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',

tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       grad_fn=<AddmmBackward>)
コンクリート
tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       grad_fn=<AddmmBackward>)
コンクリート
tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       grad_fn=<AddmmBackward>)
コンクリート
tensor([[-15.7126,  -4.5258,  14.2280,   5.9914]], device='cuda:0',
       grad_fn=<AddmmBackward>)
背景あり
tensor([[ 2.2001, -3.5031,  6.3317, -5.0941]], device='cuda:0',
       grad_fn=<AddmmBackward>)
背景あり
tensor([[-33.5647,  -5.5287,  22.0709,  17.0506]], device='cuda:0',
       grad_fn=<AddmmBackward>)
背景あり
tensor([[-27.3927,  -4.9915,  19.0499,  13.3559]], device='cuda:0',
       grad_fn=<AddmmBackward>)
背景あり
tensor([[-16.2894,  -4.4399,  14.2891,   6.4288]], device='cuda:0',
       grad_fn=<AddmmBackward>)
背景あり
tensor([[ 3.3954, -3.4524,  5.8334, -5.8457]], device='cuda:0',
       grad_fn=<AddmmBackward>)
背景あり
tensor([[-16.2651,  -4.3568,  14.1456,   6.4690]], device='cuda:0',
 

tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       grad_fn=<AddmmBackward>)
コンクリート
tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       grad_fn=<AddmmBackward>)
コンクリート
tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       grad_fn=<AddmmBackward>)
コンクリート
tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       grad_fn=<AddmmBackward>)
コンクリート
tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       grad_fn=<AddmmBackward>)
コンクリート
tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       grad_fn=<AddmmBackward>)
コンクリート
tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       grad_fn=<AddmmBackward>)
コンクリート
tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       grad_fn=<AddmmBackward>)
コンクリート
tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       grad_fn=<AddmmBackward>)
コンクリート
tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       gr

tensor([[-2143.3940, -1510.6411,  2311.1016,  1343.5974]], device='cuda:0',
       grad_fn=<AddmmBackward>)
背景あり
tensor([[-2178.9473, -1593.0729,  2417.1487,  1355.3314]], device='cuda:0',
       grad_fn=<AddmmBackward>)
背景あり
tensor([[-3620.5984, -7152.4238,  8628.4658,  2140.7866]], device='cuda:0',
       grad_fn=<AddmmBackward>)
背景あり
tensor([[-3537.1567, -8641.3896, 10095.3096,  2078.2900]], device='cuda:0',
       grad_fn=<AddmmBackward>)
背景あり
tensor([[-1296.4692,  -731.8000,  1161.3090,   868.4080]], device='cuda:0',
       grad_fn=<AddmmBackward>)
背景あり
tensor([[-1893.9727, -1256.1881,  1947.4047,  1203.6404]], device='cuda:0',
       grad_fn=<AddmmBackward>)
背景あり
tensor([[-74.4816, -56.4889,  75.0485,  56.0603]], device='cuda:0',
       grad_fn=<AddmmBackward>)
背景あり
tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       grad_fn=<AddmmBackward>)
コンクリート
tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       grad_fn=<AddmmBackward>)
コンクリート
tensor([[-1

...

tensor([[-44.1462,  -6.1231,  26.7196,  23.6057]], device='cuda:0',
       grad_fn=<AddmmBackward>)
背景あり
tensor([[ 104317.3125, -201719.5625,  210073.9688, -112883.3125]],
       device='cuda:0', grad_fn=<AddmmBackward>)
背景あり
tensor([[ 100171.8594, -195800.4844,  203837.6250, -108413.5391]],
       device='cuda:0', grad_fn=<AddmmBackward>)
背景あり
tensor([[ 22048.4277, -74639.8359,  77080.1016, -24557.1660]], device='cuda:0',
       grad_fn=<AddmmBackward>)
背景あり
tensor([[  42987.9141, -109804.0938,  113638.5391,  -46929.1133]],
       device='cuda:0', grad_fn=<AddmmBackward>)
背景あり
tensor([[ 11691.8369, -60615.8555,  62394.7695, -13523.7598]], device='cuda:0',
       grad_fn=<AddmmBackward>)
背景あり
tensor([[-225.8091,  -74.7194,  154.7426,  146.1306]], device='cuda:0',
       grad_fn=<AddmmBackward>)
背景あり
tensor([[-559.7426, -288.4024,  471.9229,  376.9961]], device='cuda:0',
       grad_fn=<AddmmBackward>)
背景あり
tensor([[-229.9034,  -83.6800,  164.5098,  149.4560]], device='cuda:0',
       g

tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       grad_fn=<AddmmBackward>)
コンクリート
tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       grad_fn=<AddmmBackward>)
コンクリート
tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       grad_fn=<AddmmBackward>)
コンクリート
tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       grad_fn=<AddmmBackward>)
コンクリート
tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       grad_fn=<AddmmBackward>)
コンクリート
tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       grad_fn=<AddmmBackward>)
コンクリート
tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       grad_fn=<AddmmBackward>)
コンクリート
tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       grad_fn=<AddmmBackward>)
コンクリート
tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       grad_fn=<AddmmBackward>)
コンクリート
tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       gr

tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       grad_fn=<AddmmBackward>)
コンクリート
tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       grad_fn=<AddmmBackward>)
コンクリート
tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       grad_fn=<AddmmBackward>)
コンクリート
tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       grad_fn=<AddmmBackward>)
コンクリート
tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       grad_fn=<AddmmBackward>)
コンクリート
tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       grad_fn=<AddmmBackward>)
コンクリート
tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       grad_fn=<AddmmBackward>)
コンクリート
tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       grad_fn=<AddmmBackward>)
コンクリート
tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       grad_fn=<AddmmBackward>)
コンクリート
tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       gr

tensor([[-54.3689,  -6.6974,  31.2107,  29.9385]], device='cuda:0',
       grad_fn=<AddmmBackward>)
背景あり
tensor([[-36.1824,  -7.5102,  20.4723,  23.2562]], device='cuda:0',
       grad_fn=<AddmmBackward>)
金枠あり
tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       grad_fn=<AddmmBackward>)
コンクリート
tensor([[-15.1326,  -4.4933,  13.9732,   5.6322]], device='cuda:0',
       grad_fn=<AddmmBackward>)
背景あり
tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       grad_fn=<AddmmBackward>)
コンクリート
tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       grad_fn=<AddmmBackward>)
コンクリート
tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       grad_fn=<AddmmBackward>)
コンクリート
tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       grad_fn=<AddmmBackward>)
コンクリート
tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
       grad_fn=<AddmmBackward>)
コンクリート
tensor([[ 8.1314, -3.1864,  3.7527, -8.7796]], device='cuda:0',
   

tensor([[-999.5334, -557.7105,  876.4336,  682.0587]], device='cuda:0',
       grad_fn=<AddmmBackward>)
背景あり
